In [1]:
import sys
print("Python executable:", sys.executable)
print("Python version:", sys.version)

Python executable: /opt/homebrew/anaconda3/envs/venv_mitsui_condapy310/bin/python
Python version: 3.10.15 | packaged by conda-forge | (main, Oct 16 2024, 01:24:20) [Clang 17.0.6 ]


In [ ]:
######################################## Install packages ########################################

import pandas as pd
import numpy as np
# Create chunks
import re

# Model for NER
import spacy 
from sklearn.cluster import KMeans
import medspacy
from medspacy.ner import TargetRule
from thefuzz import fuzz, process

#UMLSClient for NER
import umls_api
from umls_api_client import UMLS
from quickumls import QuickUMLS

# Use natural language processing (NLP) to extract keywords from the criteria
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('wordnet')
from sentence_transformers import SentenceTransformer, util

# Performance
import sklearn
from sklearn.metrics import cohen_kappa_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

import snowflake.connector

In [ ]:
######################################## Connect to Snowflake ########################################

# Establish a connection
conn = snowflake.connector.connect(
    user='dana_george@hakkoda.io',
    authenticator='externalbrowser',
    account='ska04930.east-us-2.azure',
    warehouse='DATASCIENCE_WH',
    database='ONCOEMR_RAW_DEV',
    schema='DBO',
    role='ACCOUNTADMIN'
)

# Run a test query
cursor = conn.cursor()
cursor.execute("SELECT CURRENT_VERSION()")
row = cursor.fetchone()
print("Snowflake version:", row[0])

In [ ]:
######################################## Load Data ########################################

# Get sample patient ids
cursor = conn.cursor()
cursor.execute("""
    SELECT DISTINCT "patientid" FROM ONCOEMR_RAW_DEV.DBO.DEMOGRAHPICS
    ORDER BY RANDOM()
""")
sample_patient_ids = [row[0] for row in cursor.fetchall()]
sample_patient_ids = [f"'{id}'" if isinstance(id, str) else str(id) for id in sample_patient_ids]

cursor = conn.cursor()
cursor.execute("""
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'DBO'
    AND table_catalog = 'ONCOEMR_RAW_DEV'
    AND table_type = 'BASE TABLE';
""")

# Fetch all the table names
tables = [row[0] for row in cursor.fetchall()]
#print(tables)

# Create a dictionary to hold each table as a DataFrame
table_dataframes = {}
table_dataframes_spat = {}

for table in tables:
    # First, check if the table contains 'patientid' by querying the columns of the table
    cursor.execute(f"""
        SELECT column_name
        FROM information_schema.columns
        WHERE table_schema = 'DBO' AND table_name = '{table}'
    """)
    
    columns = [row[0] for row in cursor.fetchall()]
    
    # If 'patientid' is a column, proceed to query the table
    if 'patientid' in columns:
        query = f"""
            SELECT *
            FROM ONCOEMR_RAW_DEV.DBO.{table}
            WHERE "patientid" IN ({', '.join(map(str, sample_patient_ids))})
        """
        cursor.execute(query)
        
        # Fetch the result and convert it to a DataFrame
        results = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]
        table_dataframes[table] = pd.DataFrame(results, columns=columns)

    # If 'spatientid' is a column, proceed to query the table
    if 'spatientid' in columns:
        query = f"""
            SELECT *
            FROM ONCOEMR_RAW_DEV.DBO.{table}
            WHERE "spatientid" IN ({', '.join(map(str, sample_patient_ids))})
        """
        cursor.execute(query)
        
        # Fetch the result and convert it to a DataFrame
        results_spat = cursor.fetchall()
        columns_spat = [desc[0] for desc in cursor.description]
        table_dataframes_spat[table] = pd.DataFrame(results_spat, columns=columns_spat)

# Merge table_dataframes_spat into table_dataframes_pat
table_dataframes.update(table_dataframes_spat)

# Now table_dataframes_pat contains all the tables from both dictionaries
print("Tables Loaded:")
for table, df in table_dataframes.items():
    print(f"{table}")
    #print(df.head())

# Loop through the dictionary to create a separate DataFrame variable for each key
for table, df in table_dataframes.items():
    globals()[table] = df

# Now you can access the DataFrames as individual variables:
# print(ADMINISTRATIONS.head())

# Bring in clinical trial data
clinical_trials_excl = pd.read_csv('clinical_trials_data_simple_exclusion.csv')
clinical_trials_incl = pd.read_csv('clinical_trials_data_simple_inclusion.csv')
print("clinical_trials_data_simple_exclusion")
print("clinical_trials_data_simple_inclusion")
print(" ")
print("Data Loaded Successfully!")

In [ ]:
def print_columns_of_dict_of_dfs(df_dict):
    """Prints the columns of each DataFrame in a dictionary of DataFrames."""

    for df_name, df in df_dict.items():
        print(f"Columns of {df_name}:")
        print(df.columns)
        print("-" * 20)

# Call the function to print the columns
print_columns_of_dict_of_dfs(table_dataframes)

In [ ]:
######################################## Feature Engineering ########################################

# Convert non-numeric values to NaN
DEMOGRAPHICS['age'] = pd.to_numeric(DEMOGRAPHICS['age'], errors='coerce')

# Now, convert the column to integers (NaNs will remain as NaN)
DEMOGRAPHICS['age'] = DEMOGRAPHICS['age'].fillna(-1).astype(int)  
print("Feature Engineering Complete!")

In [ ]:
######################################## Quality Check ########################################
print(clinical_trials_incl.columns)
print(DEMOGRAPHICS)

In [ ]:
######################################## Build Mock Expert Decision ########################################
# Perform the LEFT JOIN
merged_df = DEMOGRAPHICS.merge(DIAGNOSIS, on='patientid', how='left')

# Filter using "LIKE" equivalent
eligible = merged_df[
    (merged_df['age'] >= 18) &
    (merged_df['gender'] == 'Female') &
    (merged_df['targetdetaileddiagnosisgroup'].str.contains('lung', case=False, na=False)) &
    (merged_df['targetdetaileddiagnosisgroup'].str.contains('cancer', case=False, na=False))
]

# eligible['Expert Decision'] = 1

# Extract patient IDs that match the expert's eligibility criteria
eligible_patient_ids = eligible['patientid'].unique().tolist()

# Get patient IDs that are not in the eligible list
ineligible_patient_ids = merged_df[~merged_df['patientid'].isin(eligible_patient_ids)]['patientid'].unique().tolist()

# SELECT * FROM DEMOGRAPHICS
# LEFT JOIN DIAGNOSIS ON PATIENTID
# WHERE AGE >= 18 AND GENDER = 'FEMALE'
# AND DIAGNOSIS like LUNG CANCER

print(len(eligible_patient_ids))
print(len(ineligible_patient_ids))

In [ ]:
print(clinical_trials_incl)

In [ ]:
### Test to apply to 1 line of text
### Create Inclusion Criteria Categories to be used later in column matching/finding

# Load the MedSpaCy model
nlp = spacy.load('en_ner_bc5cdr_md')

# Process your text
text = "The patient is a female 18 years old and was diagnosed with breast cancer and prescribed Tamoxifen."

# Function to extract entities and labels
def extract_entities(text):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]

    # Custom check for age-related information (e.g., "18 years old")
    age_pattern = r'\b(?:aged|over|under|above|below)?\s*(\d+)\s*(?:years? old|yrs?|yo)?\b'
    age_matches = re.findall(age_pattern, text, re.IGNORECASE)
    
    # If age-related information is found, add it to the entities with the correct label
    for age in age_matches:
        entities.append((f"{age} years old", 'AGE'))
    
    # Custom check for gender-related information (e.g., "Male", "Female")
    gender_keywords = ['female', 'male']  # We only need to check for 'female' and 'male'
    
    # Check for the first gender-related term match (female first, then male)
    gender_found = False
    for gender in gender_keywords:
        match = re.search(r'\b' + gender + r'\b', text, re.IGNORECASE)
        if match:
            entities.append((match.group(), 'GENDER'))
            break  # Once a match is found, stop further checking

    return entities

# Display named entities and custom additions
entities = extract_entities(text)
for ent in entities:
    print(f"Entity: {ent[0]}, Label: {ent[1]}")

In [ ]:
### Apply to a dataframe of trial data
### Create Inclusion Criteria Categories to be used later in column matching/finding

# Load the MedSpaCy model
nlp = spacy.load('en_ner_bc5cdr_md')

# Function to extract entities and labels
def extract_entities(text):
    # Process the text through the NLP model
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]

    # Custom check for age-related information (e.g., "18 years old")
    age_pattern = r'\b(?:aged|over|under|above|below)?\s*(\d+)\s*(?:years? old|yrs?|yo)?\b'
    age_matches = re.findall(age_pattern, text, re.IGNORECASE)
    
    # If age-related information is found, add it to the entities with the correct label
    for age in age_matches:
        entities.append((f"{age[0]} years old", 'AGE'))
    
    # Custom check for gender-related information (e.g., "Male", "Female")
    gender_keywords = ['female', 'male']  # We only need to check for 'female' and 'male'
    
    # Check for the first gender-related term match (female first, then male)
    gender_found = False
    for gender in gender_keywords:
        match = re.search(r'\b' + gender + r'\b', text, re.IGNORECASE)
        if match:
            entities.append((match.group(), 'GENDER'))
            break  # Once a match is found, stop further checking

    # Extract the unique labels to avoid duplicates and return them
    unique_labels = set([label for _, label in entities])
    return list(unique_labels)

# Apply the function to the inclusion_criteria column and create a new 'Category' column
clinical_trials_incl['Category'] = clinical_trials_incl['Inclusion_Criteria'].apply(lambda x: ', '.join(extract_entities(x)))

# Display the updated DataFrame
print(clinical_trials_incl)

In [ ]:
# Function to find exact matches and fuzzy matches
def find_matching_columns(category, dict_of_dfs, fuzzy_threshold=80):
    # Step 1: Find exact matches (case-insensitive)
    exact_matches = []
    for df_name, df in dict_of_dfs.items():
        if category.lower() in [col.lower() for col in df.columns]:
            exact_column = next(col for col in df.columns if col.lower() == category.lower())
            exact_matches.append(f'{df_name}.{exact_column}')
            return exact_matches  # Return immediately after finding an exact match
    
    # Step 2: If no exact match, find fuzzy matches
    fuzzy_matches = []
    for df_name, df in dict_of_dfs.items():
        columns = df.columns
        for column in columns:
            score = process.extractOne(category, [column])  # Compare category with each column
            if score and score[1] >= fuzzy_threshold:  # If score is above threshold
                fuzzy_matches.append(f'{df_name}.{column}')
    
    return fuzzy_matches

# Loop through the clinical_trials_incl DataFrame and apply matching function
def add_source_columns(clinical_trials_incl, table_dataframes):
    source_columns_list = []
    
    for index, row in clinical_trials_incl.iterrows():
        category = row['Category']
        matching_columns = find_matching_columns(category, table_dataframes)
        
        # If there are multiple matches, list them, else return 'No match'
        if matching_columns:
            source_columns_list.append(', '.join(matching_columns))
        else:
            source_columns_list.append('No match')
    
    clinical_trials_incl['source_columns'] = source_columns_list
    return clinical_trials_incl

# Apply the function to the clinical_trials_incl DataFrame
updated_df = add_source_columns(clinical_trials_incl, table_dataframes)

# Display the updated DataFrame
print(updated_df)